In [44]:
import os
import cv2
save_folder=r'/media/cgi/440A20600A20516A/drowsy_dataset/drowsy_yolo_real'
folder_list=['0_image','10_image']
for folder in folder_list:
    folder_path=os.path.join(save_folder,folder)
    if not os.path.isdir(folder_path):
        os.makedirs(folder_path)
    else:
        print("Folder already exists.")



Folder already exists.


In [20]:
save_folder.split('/')[-1]

'drowsy_yolo_real'

In [45]:
import random
dir_list=['0','10']
save_folder=r'/media/cgi/440A20600A20516A/drowsy_dataset/drowsy_yolo_real'
pwd=os.getcwd()
for dir in dir_list:
    folder_path=os.path.join(pwd,dir)
    video_list=os.listdir(folder_path)
    for video in video_list:
        video_path=os.path.join(folder_path,video)
        cap=cv2.VideoCapture(video_path)
        num_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        random_list=[]
        frame_count=0
        while frame_count<300:
            random_frame = random.randint(1,num_frames)
            while random_frame in random_list:
                random_frame=random.randint(1,num_frames)
                
            cap.set(cv2.CAP_PROP_POS_FRAMES,random_frame)
            ret,frame = cap.read()
            current_frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            # print(current_frame_number)
            if not ret:
                break
            image_name=f'{video}_{current_frame_number}.png'
            save_dir_path=os.path.join(save_folder,f'{dir}_image')
            save_image_path=os.path.join(save_dir_path,image_name)
            # print(save_image_path)
            cv2.imwrite(save_image_path,frame)
            frame_count+=1
            random_list.append(random_frame)

        cap.release()


        # print(video_path)

print("Execution Complete!")
        
    
   


Execution Complete!


#### Making a YOLO .txt file for the randomly selected images from the video dataset

In [48]:
import os
import cv2
save_folder=r'/media/cgi/440A20600A20516A/drowsy_dataset/drowsy_label'
os.makedirs(save_folder,exist_ok=True)
folder_list=['0_image','10_image']
subfolder_list=['images','labels']
for folder in folder_list:
    folder_path=os.path.join(save_folder,folder)
    if not os.path.isdir(folder_path):
        os.makedirs(folder_path)
    else:
        print("Folder already exists.")
    for subfolder in subfolder_list:
        subfolder_path=os.path.join(folder_path,subfolder)
        if not os.path.isdir(subfolder_path):
            os.makedirs(subfolder_path)
        else:
            print("Sub-Folder already exists.")

    

Folder already exists.
Folder already exists.


In [150]:
from ultralytics import YOLO
import torch
model_path=r'/media/cgi/440A20600A20516A/drowsy_dataset/yolov8n-face.pt'
model=YOLO(model_path)
device='cuda' if torch.cuda.is_available() else 'cpu'
model=model.to(device)
image_path=r'/media/cgi/440A20600A20516A/drowsy_dataset/hand.jpeg'
img=cv2.imread(image_path)
# img=cv2.resize(img,(640,480))
cv2.imshow('test',img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [151]:
results=model.predict(img,conf=0.6,device='cuda')


0: 640x640 (no detections), 3.3ms
Speed: 0.9ms preprocess, 3.3ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)


In [89]:
img.shape

(148, 192, 3)

In [101]:
((14.3734+44.9790)/2)/148

0.20051486486486486

In [96]:
yolo_list

[[0.33845437, 0.200515, 0.11958617, 0.20679449],
 [0.6986775, 0.3891679, 0.10678264, 0.1880054]]

In [93]:
xc,yc,w,h

(0.6986775, 0.3891679, 0.10678264, 0.1880054)

In [149]:
result=results[0]
yolo_list=[]
for box in result.boxes:
    x0,y0,x1,y1=box.xyxy.cpu().numpy()[0].astype(int)
    xc,yc,w,h=box.xywhn.cpu().numpy()[0]
    yolo_list.append(f"test {xc} {yc} {w} {h}\n")

    print(x0,y0,x1,y1)
    img = cv2.rectangle(img,(x0,y0), (x1,y1),(0,255,0),2)
    results=model.predict(img,conf=0.6,device='cuda')
    
    # with open('my_file.txt', 'w') as f:
    #     f.writelines(yolo_list)
    #     print("Successfully wrote to the file!")

    cv2.imshow('test',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


106 37 152 100


#### Writing the .txt file into the folder

label format = 0 0.5168269230769231 0.4735576923076923 0.12259615384615384 0.03365384615384615
                class_number xc yc w h 

In [ ]:
from ultralytics import YOLO
import torch
import shutil

model_path=r'/media/cgi/440A20600A20516A/drowsy_dataset/yolov8n-face.pt'
model=YOLO(model_path)
device='cuda' if torch.cuda.is_available() else 'cpu'
model=model.to(device)

move_folder_path=r'/media/cgi/440A20600A20516A/drowsy_dataset/drowsy_yolo_real'
save_folder_path=r'/media/cgi/440A20600A20516A/drowsy_dataset/drowsy_label'
move_folders=os.listdir(move_folder_path)
for folder in move_folders:
    folder_path=os.path.join(move_folder_path,folder)
    save_subfolder=os.path.join(save_folder_path,folder)
    save_image_path=os.path.join(save_subfolder,'images')
    save_label_path=os.path.join(save_subfolder,'labels')
    class_name=int(folder.split("_")[0][0])
    file_list=os.listdir(folder_path)
    for file in file_list:
        move_file_path=os.path.join(folder_path,file)
        file_name=file.split('.png')[0]
        label_name=f'{file_name}.txt'
        image_destination_path=os.path.join(save_image_path,file)
        label_destination_path=os.path.join(save_label_path,label_name)
        
        img=cv2.imread(move_file_path)
        results=model.predict(img,conf=0.6,device='cuda')
        result=results[0]
        try:
            if len(result.boxes) !=1:
                continue

            label_list=[]
            for box in result.boxes:
                xc,yc,w,h=box.xywhn.cpu().numpy()[0]
                label_list.append(f"{class_name} {xc} {yc} {w} {h}\n")
        
            with open(label_destination_path, 'w') as f:   
                f.writelines(label_list)
            f.close()
            # shutil.copy(source_path, destination_path) (Syntax to use shutil)
            shutil.copy(move_file_path,image_destination_path)
        except:
            print(len(result.boxes))
            print("An error occured!")



#### Creating the YOLO dataset 

In [1]:
import os

yolo_folder=r'/media/cgi/440A20600A20516A/drowsy_dataset/uta_rldd_yolo'
os.makedirs(yolo_folder,exist_ok=True)
folder_list=['train','test','valid']
subfolder_list=['images','labels']
for folder in folder_list:
    folder_path=os.path.join(yolo_folder,folder)
    if not os.path.isdir(folder_path):
        os.makedirs(folder_path)
    else:
        print("Folder already exists.")
    for subfolder in subfolder_list:
        subfolder_path=os.path.join(folder_path,subfolder)
        if not os.path.isdir(subfolder_path):
            os.makedirs(subfolder_path)
        else:
            print("Sub-Folder already exists.")

Folder already exists.
Sub-Folder already exists.
Sub-Folder already exists.
Folder already exists.
Sub-Folder already exists.
Sub-Folder already exists.
Folder already exists.
Sub-Folder already exists.
Sub-Folder already exists.


In [23]:
import random
import shutil

move_folder_path=r'/media/cgi/440A20600A20516A/drowsy_dataset/drowsy_label'
save_folder_path=r'/media/cgi/440A20600A20516A/drowsy_dataset/uta_rldd_yolo'
move_folders=os.listdir(move_folder_path)
train_list=[]
val_list=[]
test_list=[]

#Setting the train folder path
train_folder_path=os.path.join(save_folder_path,'train')
train_image_path=os.path.join(train_folder_path,'images')
train_label_path=os.path.join(train_folder_path,'labels')

#Setting the valid folder path
valid_folder_path=os.path.join(save_folder_path,'valid')
valid_image_path=os.path.join(valid_folder_path,'images')
valid_label_path=os.path.join(valid_folder_path,'labels')

#Setting the test folder path
test_folder_path=os.path.join(save_folder_path,'test')
test_image_path=os.path.join(test_folder_path,'images')
test_label_path=os.path.join(test_folder_path,'labels')

for folder in move_folders:
    number_done_list=[]
    image_count=0
    folder_path=os.path.join(move_folder_path,folder)
    subfolder_path=os.path.join(folder_path,'images')
    file_list=os.listdir(subfolder_path)
    total_images=len(file_list)
    train_split=int(0.7*total_images)
    val_split=int(0.15*total_images)
    test_split=total_images-train_split-val_split
    file_num=[i for i in range(len(file_list))]

    move_folder_class=os.path.join(move_folder_path,folder)  #move_folder_path + /0_image
    move_folder_images=os.path.join(move_folder_class,'images') #move_folder_path + /0_image/images
    move_folder_labels=os.path.join(move_folder_class,'labels')

    while image_count<train_split:
        random_number = random.randint(0,total_images-1)
        while random_number in number_done_list:
            random_number=random.randint(0,total_images-1)
        
        image_file=file_list[random_number]
        label_file=file_list[random_number].split('.png')[0]+'.txt'

        move_file_path=os.path.join(move_folder_images,image_file)  #move_folder_path +0_image/images/<image_path>
        move_label_path=os.path.join(move_folder_labels,label_file)

        train_image_file=os.path.join(train_image_path,image_file)
        train_label_file=os.path.join(train_label_path,label_file)

        # shutil.copy(source_path, destination_path) (Syntax to use shutil)
        shutil.copy(move_file_path,train_image_file)
        shutil.copy(move_label_path,train_label_file)

        train_list.append(file_list[random_number])
        number_done_list.append(random_number)
        image_count+=1

    image_count=0  #Setting it to 0 to get the files for the val folder
    while image_count<val_split:
        random_number = random.randint(0,total_images-1)
        while random_number in number_done_list:
            random_number=random.randint(0,total_images-1)

        image_file=file_list[random_number]
        label_file=file_list[random_number].split('.png')[0]+'.txt'

        move_file_path=os.path.join(move_folder_images,image_file)  #move_folder_path +0_image/images/<image_path>
        move_label_path=os.path.join(move_folder_labels,label_file)

        valid_image_file=os.path.join(valid_image_path,image_file)
        valid_label_file=os.path.join(valid_label_path,label_file)

        shutil.copy(move_file_path,valid_image_file)
        shutil.copy(move_label_path,valid_label_file)

        val_list.append(file_list[random_number])
        number_done_list.append(random_number)
        image_count+=1

    test_num=list(set(file_num)-set(number_done_list))
    for num in test_num:
        image_file=file_list[num]
        label_file=file_list[num].split('.png')[0]+'.txt'

        move_file_path=os.path.join(move_folder_images,image_file)  #move_folder_path +0_image/images/<image_path>
        move_label_path=os.path.join(move_folder_labels,label_file)

        test_image_file=os.path.join(test_image_path,image_file)
        test_label_file=os.path.join(test_label_path,label_file)

        shutil.copy(move_file_path,test_image_file)
        shutil.copy(move_label_path,test_label_file)
       
        test_list.append(file_list[num])


#### Checking if the folders were created correctly

In [20]:
import os
folder_path=r'/media/cgi/440A20600A20516A/drowsy_dataset/uta_rldd_yolo'
folder_list=os.listdir(folder_path)
train_list=[]
val_list=[]
test_list=[]
for folder in folder_list:
    new_folder=os.path.join(folder_path,folder)
    # print(new_folder)
    subfolder_list=os.listdir(new_folder)
    label_folder_path=os.path.join(new_folder,'labels')
    label_files_png=set([file.split('.txt')[0]+'.png' for file in os.listdir(label_folder_path)])
    image_folder_path=os.path.join(new_folder,'images')
    image_list=set(os.listdir(image_folder_path))

    if folder =='train':
        train_list=os.listdir(image_folder_path).copy()
    elif folder=='valid':
        val_list=os.listdir(image_folder_path).copy()
    else:
        test_list=os.listdir(image_folder_path).copy()

    if len(label_files_png-image_list)!=0:
        print("Some error occured")
    else:
        print("Dataset looks fine!")


print(set(train_list).isdisjoint(test_list))
print(set(train_list).isdisjoint(val_list))
print(set(test_list).isdisjoint(val_list))




Dataset looks fine!
Dataset looks fine!
Dataset looks fine!
True
True
True


In [19]:
folder_list

['test', 'train', 'valid']

In [16]:
if len(set([1,2,3])-set([1,2,3]))==0:
    print("Moye Moye")

Moye Moye


In [7]:
test=os.listdir(label_folder_path)

In [10]:
test[0].split('.txt')[0]+'.png'

'01_0.mov_10220.png'

In [4]:
subfolder_list

['images', 'labels']

In [22]:
file_list[0].split('.png')[0]+'.txt'

'01_10.MOV_10441.txt'

In [19]:
move_folders

['0_image', '10_image']

In [18]:
test_image_path

'/media/cgi/440A20600A20516A/drowsy_dataset/uta_rldd_yolo/test/images'

In [8]:
len(train_list),len(val_list),len(test_list)

(12946, 2773, 2777)

In [13]:
set(val_list).isdisjoint(set(test_list))

True

In [10]:
len(set(train_list)-set(val_list))

12946

In [192]:
file_list[0]

'01_10.MOV_10441.png'

In [191]:
train_split,test_split,val_split

(6885, 1477, 1475)

In [188]:
int(len(file_list)*0.7)

6885

In [172]:
if result.boxes is None:
    print("hihiho")

In [144]:
label_destination_path

'/media/cgi/440A20600A20516A/drowsy_dataset/drowsy_label/0_image/labels/24_0.mp4_2152.txt'

In [141]:
file.split('.png')[0]

'24_0.mp4_2152'

In [124]:
new_string

'10 0.05 0.06 0.07 0.08\n'

In [109]:
move_folders[0].split("_")[0]

'0'

In [56]:
result=model

(1920, 1080, 3)

In [ ]:
zero_path=r'/media/cgi/440A20600A20516A/drowsy_dataset/0'
